<a href="https://colab.research.google.com/github/dhruv0000/neural-robot-dynamics/blob/main/train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Robot Dynamics Training on Colab

This notebook demonstrates how to setup the environment, generate a dataset, and train the NeRD model.

In [10]:
# 1. Setup Environment
!git clone https://github.com/dhruv0000/neural-robot-dynamics.git
%cd neural-robot-dynamics
!pip install -r requirements.txt
!pip install warp-lang==1.8.0
!pip install rl_games

Cloning into 'neural-robot-dynamics'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (466/466), done.
remote: Compressing objects: 100% (340/340), done.
remote: Total 466 (delta 115), reused 431 (delta 82), pack-reused 0 (from 0)
Receiving objects: 100% (466/466), 17.30 MiB | 28.85 MiB/s, done.
Resolving deltas: 100% (115/115), done.
Filtering content: 100% (11/11), 202.03 MiB | 88.64 MiB/s, done.
/content/neural-robot-dynamics/train/neural-robot-dynamics/neural-robot-dynamics
  Using cached pyglet-2.1.6-py3-none-any.whl.metadata (7.7 kB)
  Using cached ipdb-0.13.13-py3-none-any.whl.metadata (14 kB)
  Using cached h5py-3.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached tensorboard-2.14.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached matplotlib-3.7.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.wh

In [ ]:
# 2. Generate Dataset
# We generate a smaller dataset for demonstration purposes.
import os
import shutil
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

%cd generate

# Define paths
drive_data_dir = '/content/drive/MyDrive/neural-robot-dynamics/data/datasets/'
local_data_dir = '../data/datasets/'
train_filename = 'trajectory_len-100_train.hdf5'
valid_filename = 'trajectory_len-100_valid.hdf5'

os.makedirs(local_data_dir, exist_ok=True)
os.makedirs(drive_data_dir, exist_ok=True)

# Check if data exists in Drive
if os.path.exists(os.path.join(drive_data_dir, train_filename)) and os.path.exists(os.path.join(drive_data_dir, valid_filename)):
    print("Loading datasets from Google Drive...")
    shutil.copy(os.path.join(drive_data_dir, train_filename), local_data_dir)
    shutil.copy(os.path.join(drive_data_dir, valid_filename), local_data_dir)
else:
    print("Generating datasets...")
    # Generate Training Data
    !python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 10000000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_train.hdf5 --trajectory-length 100 --num-envs 64 --seed 0

    # Generate Validation Data
    !python generate_dataset_contact_free.py --env-name Cartpole --num-transitions 100000 --dataset-dir ../data/datasets/ --dataset-name trajectory_len-100_valid.hdf5 --trajectory-length 100 --num-envs 64 --seed 10

    print("Saving datasets to Google Drive...")
    shutil.copy(os.path.join(local_data_dir, train_filename), drive_data_dir)
    shutil.copy(os.path.join(local_data_dir, valid_filename), drive_data_dir)

%cd ..

/content/neural-robot-dynamics/train/neural-robot-dynamics/neural-robot-dynamics/generate
Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
 [NeuralEnvironment] Creating abstract contact environment: Cartpole. 
Creating 64 environments: 100% 64/64 [00:00<00:00, 187.85it/s]
Module warp.sim.integrator_featherstone 18b3327 load on device 'cuda:0' took 21162.08 ms  (compiled)
Module envs.abstract_contact_environment 8e8d790 load on device 'cuda:0' took 570.62 ms  (compiled)
Module integrators.integrator_neural ee402cd load on device 'cuda:0' took 743.02 ms  (compiled)
 [NeuralEnvironment] Created a DUMMY Neural Integrator. 
  0% 0/10000 [00:00<?, ?it/s]Module utils.warp_utils 294c46

In [12]:
# 3. Train Baseline Model (Transformer)
%cd train

import yaml
import os

# Load default config
with open('cfg/Cartpole/transformer.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Set num_envs to 64 for Colab T4
cfg['env']['num_envs'] = 64

# Override dataset paths to point to the generated data
cfg['algorithm']['dataset']['train_dataset_path'] = '../data/datasets/Cartpole/trajectory_len-100_train.hdf5'
cfg['algorithm']['dataset']['valid_datasets']['exp_trajectory'] = '../data/datasets/Cartpole/trajectory_len-100_valid.hdf5'

# Reduce training parameters for quick demonstration
# cfg['algorithm']['num_epochs'] = 5
 # Use default 1000 epochs from paper
# cfg['algorithm']['num_iters_per_epoch'] = 100
 # Use default 5000 iters from paper

# Save the modified config
with open('colab_config.yaml', 'w') as f:
    yaml.dump(cfg, f)

# Run training
!python train.py --cfg colab_config.yaml --logdir ../data/logs/baseline

/content/neural-robot-dynamics/train/neural-robot-dynamics/neural-robot-dynamics/train
Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-24 16:37:08.685047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764002228.704545    3187 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764002228.710463    3187 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already bee

In [13]:
# 4. Train Mamba Model
# We use the same config but add the --novelty mamba flag
# !python train.py --cfg colab_config.yaml --novelty mamba --logdir ../data/logs/mamba

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-24 16:38:24.598209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764002304.630227    3974 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764002304.640271    3974 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764002304.655190    3974 computation_placer.cc:177] computati

In [14]:
# 5. Train Unroll Model
# We use the same config but add the --novelty unroll flag
# !python train.py --cfg colab_config.yaml --novelty unroll --logdir ../data/logs/unroll

Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
2025-11-24 16:42:17.699431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764002537.719193    5382 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764002537.725176    5382 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764002537.740513    5382 computation_placer.cc:177] computati

In [ ]:
# 6. Evaluate Models

Evaluating Baseline Model...
Using model: ../data/logs/baseline/11-24-2025-16-37-15/nn/best_eval_model.pt
Evaluating Mamba Model...
Using model: ../data/logs/mamba/11-24-2025-16-38-31/nn/best_eval_model.pt
Evaluating Unroll Model...
Using model: ../data/logs/unroll/11-24-2025-16-42-24/nn/best_eval_model.pt


# 7. Quantitative Analysis

We now perform the quantitative analysis as described in the paper experiments.
We evaluate:
1. **Long-Horizon Passive Motion**: Accuracy of the trained NeRD models over 100, 500, and 1000 steps.
2. **RL Policy Evaluation**: Performance of the pretrained RL policy using the NeRD models.

In [16]:
# 7.1 Long-Horizon Passive Motion Evaluation
# We evaluate the Baseline, Mamba, and Unroll models on Cartpole for 100, 500, and 1000 steps.

import os
import glob

def find_latest_model(model_type):
    base_log_dir = f'../data/logs/{model_type}'
    if not os.path.exists(base_log_dir):
        return None
    dirs = [d for d in glob.glob(os.path.join(base_log_dir, '*')) if os.path.isdir(d)]
    if not dirs:
        return None
    latest_dir = sorted(dirs)[-1]
    model_path = os.path.join(latest_dir, 'nn', 'best_eval_model.pt')
    if not os.path.exists(model_path):
        return None
    return model_path

models = ['baseline']
horizons = [100, 500]

for model_name in models:
    model_path = find_latest_model(model_name)
    if not model_path:
        print(f"Skipping {model_name} (model not found)")
        continue

    print(f"\n{'='*20} Evaluating {model_name.capitalize()} Model {'='*20}")
    for horizon in horizons:
        print(f"\n--- Horizon: {horizon} ---")
        # We use !python to ensure output is printed to the cell
        !python ../eval/eval_passive/eval_passive_motion.py \
            --env-name Cartpole \
            --model-path {model_path} \
            --env-mode neural \
            --num-envs 2048 \
            --num-rollouts 2048 \
            --rollout-horizon {horizon} \
            --seed 500


==================== Evaluating Baseline Model ====================

--- Horizon: 100 ---
Warp DeprecationWarning: The `warp.sim` module is deprecated and will be removed in v1.10. Please transition to using the forthcoming Newton library instead.
Warp 1.8.0 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.8.0
Number of Model Parameters:  2713668
 [NeuralEnvironment] Creating abstract contact environment: Cartpole. 
Creating 2048 environments: 100% 2048/2048 [00:07<00:00, 277.37it/s]
Module warp.sim.integrator_featherstone 18b3327 load on device 'cuda:0' took 3.60 ms  (cached)
Module envs.abstract_contact_environment 8e8d790 load on device 'cuda:0' took 0.33 ms  (cached)
Module integrators.integrator_neural ee402cd load on device 'cuda:0' took 0.45 ms  (cached)
 [NeuralEnvironment] Created a Neural Integrator. 
Sampling state transitions:   0% 0/

In [18]:
# 7.2 RL Policy Evaluation (Quantitative)
# We evaluate the policy using the trained NeRD models and compare with Ground Truth.
# We run for more games (2048) to get a statistically significant result as in the paper.

import pandas as pd
import re
import subprocess
import os

def run_eval(model_path=None, env_mode='neural', label='Model'):
    print(f"\n{'='*20} RL Evaluation: {label} {'='*20}")

    # Use absolute paths
    abs_playback_path = os.path.abspath('../pretrained_models/RL_policies/Cartpole/0/nn/CartpolePPO.pth')
    abs_rl_cfg_path = os.path.abspath('../eval/eval_rl/cfg/Cartpole/cartpole.yaml')

    cmd = [
        'python', 'run_rl.py',
        '--rl-cfg', abs_rl_cfg_path,
        '--playback', abs_playback_path,
        '--num-envs', '2048',
        '--num-games', '2048',
        '--env-mode', env_mode
    ]

    if model_path:
        abs_model_path = os.path.abspath(model_path)
        cmd.extend(['--nerd-model-path', abs_model_path])

    try:
        result = subprocess.run(cmd, cwd='../eval/eval_rl', check=True, capture_output=True, text=True)
        output = result.stdout
        print(output[-500:]) # Print last 500 chars to see result

        # Parse reward
        # Look for 'av reward: <value> av steps: <value>'
        match = re.search(r'av reward:\s*([-\d\.]+)', output)
        if match:
            reward = float(match.group(1))
            return reward
        else:
            print("Could not parse reward from output.")
            return None
    except subprocess.CalledProcessError as e:
        print(f'Error running RL evaluation for {label}:')
        print('STDOUT:', e.stdout)
        print('STDERR:', e.stderr)
        return None

results = []

# 1. Evaluate Ground Truth
gt_reward = run_eval(env_mode='ground-truth', label='Ground Truth')
if gt_reward is not None:
    results.append({'Model': 'Ground Truth', 'Reward': gt_reward, 'Error (%)': 0.0})

# 2. Evaluate NeRD Models
for model_name in models:
    model_path = find_latest_model(model_name)
    if not model_path:
        continue

    reward = run_eval(model_path=model_path, env_mode='neural', label=model_name.capitalize())

    if reward is not None and gt_reward is not None:
        error = (reward - gt_reward) / gt_reward * 100
        results.append({'Model': model_name.capitalize(), 'Reward': reward, 'Error (%)': error})
    elif reward is not None:
        results.append({'Model': model_name.capitalize(), 'Reward': reward, 'Error (%)': float('nan')})

# 3. Create Table
df = pd.DataFrame(results)
print("\nFinal Evaluation Comparison:")
display(df)


==================== RL Evaluation: Ground Truth ====================
teps: 56.0
reward: 98.16730499267578 steps: 61.0
reward: 57.46388244628906 steps: 62.0
reward: -94.28440856933594 steps: 94.0
reward: -85.1441421508789 steps: 100.0
reward: 1225.2821979114867 steps: 300.0
2461286.3829221725
av reward: 1201.799991661217 av steps: 295.4072265625
visited states range:
State 0: [-4.084733486175537, 4.017707347869873]
State 1: [-3.1415224075317383, 3.1415481567382812]
State 2: [-10.378302574157715, 8.506925582885742]
State 3: [-10.652484893798828, 9.791386604309082]


==================== RL Evaluation: Baseline ====================
ps: 293.0
reward: 1224.8743489583333 steps: 296.0
reward: 1259.690673828125 steps: 297.0
reward: 1267.3876953125 steps: 298.0
reward: 1273.3040771484375 steps: 299.0
reward: 1090.1022199277234 steps: 300.0
2155489.165640831
av reward: 1050.9454732524773 av steps: 290.6616284739152
visited states range:
State 0: [-4.109048366546631, 3.390089750289917]
State 1:

,Model,Reward,Error (%)
0,Ground Truth,1201.799992,0.000000
1,Baseline,1050.945473,-12.552381
2,Mamba,1090.435050,-9.266512
3,Unroll,-191.580636,-115.941141
